# Selenium을 이용하여 당일 웹툰 리스트를 카페 게시판에 올리기

In [216]:
from selenium import webdriver
from bs4 import BeautifulSoup as BS

driver = webdriver.Chrome('../../../driver/chromedriver')

In [217]:
# 웹툰 리스트 크롤링

driver.get('http://comic.naver.com/webtoon/weekday.nhn')
html = driver.page_source
bsObj = BS(html, 'html.parser')
titles = bsObj.select('div.col_selected > div.col_inner > ul > li > a')

def toon_list(titles):
    web_toon = ""
    for title in titles:
        web_toon += str(title.text+'  '+'http://comic.naver.com'+title.get('href')+'\n')
    return web_toon

In [218]:
from getpass import getpass

# 네이버 로그인
driver.get('http://www.naver.com')

id = input('id: ')
pw = getpass('pw: ')

driver.implicitly_wait(10)
tag_id = driver.find_element_by_id('id')
tag_id.send_keys(id)
tag_pw = driver.find_element_by_id('pw')
tag_pw.send_keys(pw)
tag_pw.submit()

In [219]:
## 카페로 이동
driver.get('https://m.cafe.naver.com/askdjango')

In [220]:
## 글씨기 메뉴 클릭
from selenium.webdriver.support.ui import Select

driver.find_element_by_css_selector('a[class*=WriteArticle]').click()

In [221]:
## 게시판 선택
select = Select(driver.find_element_by_name('menuid'))
select.select_by_index(1)

## 제목 쓰기 (+ 현재 날짜)
import datetime

now = datetime.datetime.now()
date_now = now.strftime('%Y-%m-%d')

driver.find_element_by_name('subject').send_keys(date_now,'네이버 웹툰 리스트')

In [222]:
## 내용 쓰기
iframe = driver.find_element_by_id('frame')
driver.switch_to_frame(iframe)

body = driver.find_element_by_id('body')
body.send_keys(toon_list(titles))

In [223]:
driver.switch_to_default_content()
driver.find_element_by_css_selector('[class*=Submit]').click()

In [224]:
driver.close()